In [ ]:
import libraries
try:
#   %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
import seaborn as sns
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from sklearn.utils import resample
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout, LSTM, Bidirectional
from sklearn.preprocessing import LabelEncoder
from keras import metrics
print(tf.__version__)


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
column_names = ['label', 'text']
train_dataset = pd.read_csv(
    "train-data.tsv",
    sep = '\t',
    names = column_names
    )
test_dataset = pd.read_csv(
    "valid-data.tsv",
    sep = '\t',
    names = column_names)

data = train_dataset.append(test_dataset, ignore_index = True)

ham_messages = train_dataset[train_dataset.label == "ham"]
spam_messages  = train_dataset[train_dataset.label == "spam"]

ham_msg_text = " ".join(ham_messages.text.to_numpy().tolist())
spam_msg_text = " ".join(spam_messages.text.to_numpy().tolist())

# wordcloud of ham messages
ham_msg_cloud = WordCloud(width =520, height =260, stopwords=STOPWORDS,max_font_size=50, background_color ="black", colormap='Blues').generate(ham_msg_text)
plt.figure(figsize=(16,10))
plt.imshow(ham_msg_cloud, interpolation='bilinear')
plt.axis('off') # turn off axis
plt.show()

# wordcloud of spam messages
spam_msg_cloud = WordCloud(width =520, height =260, stopwords=STOPWORDS,max_font_size=50, background_color ="black", colormap='Blues').generate(spam_msg_text)
plt.figure(figsize=(16,10))
plt.imshow(spam_msg_cloud, interpolation='bilinear')
plt.axis('off') # turn off axis
plt.show()

In [ ]:
def preprocessing(dataset):

  max_len = 50
  trunc_type = "post"
  padding_type = "post"
  oov_tok = "<OOV>"
  vocab_size = 500

  tokenizer = Tokenizer(num_words = vocab_size, char_level=False, oov_token = oov_tok)
  tokenizer.fit_on_texts(dataset)

  sequences = tokenizer.texts_to_sequences(dataset)
  sequences_padded = pad_sequences (sequences, maxlen = max_len, padding = padding_type, truncating = trunc_type )

  return sequences_padded

labels = data["label"]
texts = data["text"]
num_max = 1000
tok = Tokenizer(num_words = num_max)
tok.fit_on_texts(texts)
max_len = 100
x_train = tok.texts_to_sequences(train_dataset["text"])
x_test = tok.texts_to_sequences(test_dataset["text"])

train = pad_sequences(x_train, maxlen = max_len)
test = pad_sequences(x_test, maxlen = max_len)

le = LabelEncoder()
labeltrain = train_dataset["label"]
traintags = le.fit_transform(labeltrain)
labeltest = test_dataset["label"]
testtags = le.fit_transform(labeltest)

max_features = 5000
batch_size = 32
dropout = 0.2
recurrent_dropout = 0.2
epochs = 10

model = tf.keras.Sequential()
model.add(
    Embedding(
        max_features,
        128
        )
    )
model.add(
    LSTM(
        128,
        dropout = dropout,
        recurrent_dropout = recurrent_dropout,
        activation = 'relu'
        )
    )
model.add(
    Dense(
        1,
        activation = 'sigmoid'
        )
    )

model.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = [
        'acc',
        metrics.binary_accuracy
        ]
    )
model.summary()

model.fit(
    train,
    traintags,
    batch_size = batch_size,
    epochs = epochs,
    verbose = 1,
    validation_split = 0.3)

s = model.evaluate(test, testtags)
print('Loss: {:.3f}, Accuracy: {:.3f}'.format(s[0], s[1]))

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  inputstr = tok.texts_to_sequences([pred_text])
  pred = model.predict(inputstr)[0][0]

  if pred >= 0.5:
    label = "spam"
  else:
    label = "ham"

  output = [pred, label]
  return output


pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
